<a href="https://colab.research.google.com/github/cseybold/Weather/blob/main/Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
pip install meteostat

In [56]:
pip install geopy

In [57]:
import ee
from google.colab import auth
auth.authenticate_user(project_id='weatherproj-427900')

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='weatherproj-427900')

In [58]:
# Import Meteostat library and dependencies
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations
import pandas as pd
import xarray as xr
from geopy.distance import geodesic

firedf = pd.read_csv('/content/drive/MyDrive/WFIGS_inc_loc.csv')


In [59]:
firedf = firedf[['IncidentSize', 'FireDiscoveryDateTime', 'IncidentTypeCategory', 'InitialLatitude', 'InitialLongitude', 'POOState']]
firedf['FireDiscoveryDateTime'] = pd.to_datetime(firedf['FireDiscoveryDateTime'])
firedf.dropna(subset=['InitialLatitude', 'InitialLongitude'], inplace=True)
cafire = firedf[firedf['POOState'] == 'US-CA']
cafire = cafire[cafire['IncidentTypeCategory'] == 'WF']
cafire = cafire[cafire['IncidentSize'] > 1]
cafire.tail()

,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState
309485,1.80,2024-06-25 21:15:15,WF,35.561729,-120.516991,US-CA
309497,1.25,2024-06-25 18:04:39,WF,41.759312,-122.490208,US-CA
309520,11.03,2024-06-25 23:30:19,WF,38.845861,-123.019703,US-CA
309569,12.00,2024-06-26 02:28:04,WF,36.631882,-119.322600,US-CA
309592,53.91,2024-06-26 05:00:46,WF,37.732422,-120.596351,US-CA


In [60]:
num_NAs = cafire['FireDiscoveryDateTime'].isna().sum()
print(num_NAs)
cafire = cafire.reset_index(drop=True)
cafire.tail()

0


,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState
3811,1.80,2024-06-25 21:15:15,WF,35.561729,-120.516991,US-CA
3812,1.25,2024-06-25 18:04:39,WF,41.759312,-122.490208,US-CA
3813,11.03,2024-06-25 23:30:19,WF,38.845861,-123.019703,US-CA
3814,12.00,2024-06-26 02:28:04,WF,36.631882,-119.322600,US-CA
3815,53.91,2024-06-26 05:00:46,WF,37.732422,-120.596351,US-CA


In [61]:
testdf = cafire.head(50)

def weather_stats(lat, lon, time):
    # Create a Point for the location
    location = Point(lat, lon)

    # Get hourly data
    data = Daily(location, time, time + timedelta(days=1))
    data = data.fetch()

    # Check if data is empty
    if data.empty:
        # Find the nearest station with data
        stations = Stations()
        stations = stations.nearby(lat, lon)
        station = stations.fetch(1)

        if not station.empty:
            location = Point(station.iloc[0]['latitude'], station.iloc[0]['longitude'])
            data = Daily(location, time, time + timedelta(days=1))
            data = data.fetch()

    # Return the temperature, precipitation, and wind speed
    if not data.empty:
        return [data.iloc[0]['tavg'], data.iloc[0]['prcp'], data.iloc[0]['wspd']]
    else:
        return None

# Apply the function to each row and handle None values
def apply_weather_stats(row, i):
    stats = weather_stats(row['InitialLatitude'], row['InitialLongitude'], row['FireDiscoveryDateTime'])
    return stats[i] if stats is not None else None

testdf['Temperature'] = testdf.apply(lambda row: apply_weather_stats(row, 0), axis=1)
testdf['Precipitation'] = testdf.apply(lambda row: apply_weather_stats(row, 1), axis=1)
testdf['WindSpeed'] = testdf.apply(lambda row: apply_weather_stats(row, 2), axis=1)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [62]:
testdf['Precipitation'].fillna(0, inplace=True)
testdf.dropna(subset=['Temperature'], inplace=True)
testdf.dropna(subset=['WindSpeed'], inplace=True)

testdf.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Temperature,Precipitation,WindSpeed
44,1.5,2019-09-05 23:17:55,WF,34.448900,-118.056300,US-CA,29.6,0.0,15.8
45,80.0,2022-06-21 16:48:36,WF,33.546326,-116.043868,US-CA,32.9,0.0,12.4
46,2.0,2018-06-16 16:30:00,WF,34.665230,-118.761900,US-CA,9.3,0.0,25.6
48,350.0,2019-10-10 19:56:55,WF,33.984818,-117.217944,US-CA,19.9,0.0,13.3
49,2.0,2017-07-26 00:46:09,WF,41.675830,-123.420700,US-CA,13.1,0.0,11.3


In [63]:
def get_vegetation_index(longitude, latitude, date):
    # Define the dataset
    dataset = ee.ImageCollection('NOAA/VIIRS/001/VNP13A1')

    # Filter the dataset by date and location
    filtered_dataset = dataset.filterDate(date, date + timedelta(days = 40)).filterBounds(ee.Geometry.Point([longitude, latitude]))

    # Select 'EVI' and calculate the mean
    mean_evi = filtered_dataset.select('EVI2').mean()

    # Define a region of interest
    roi = ee.Geometry.Rectangle([longitude-0.01, latitude-0.01, longitude+0.01, latitude+0.01])

    # Reduce the image over the region of interest
    mean_evi = mean_evi.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30)

    # Check if 'EVI2' exists in the dictionary
    if 'EVI2' in mean_evi.getInfo():
        mean_evi_value = mean_evi.get('EVI2').getInfo()
    else:
        # Find the nearest station with data
        stations = Stations()
        stations = stations.nearby(latitude, longitude)
        station = stations.fetch(1)

        if not station.empty:
            nearby_lon = station.iloc[0]['longitude']
            nearby_lat = station.iloc[0]['latitude']
            nearby_roi = ee.Geometry.Rectangle([nearby_lon-0.01, nearby_lat-0.01, nearby_lon+0.01, nearby_lat+0.01])
            mean_evi = mean_evi.reduceRegion(reducer=ee.Reducer.mean(), geometry=nearby_roi, scale=30)
            if 'EVI2' in mean_evi.getInfo():
                mean_evi_value = mean_evi.get('EVI2').getInfo()
            else:
                mean_evi_value = None  # No data found even at the nearest station
        else:
            mean_evi_value = None # No nearby stations found

    return mean_evi_value

testdf['VegetationIndex'] = testdf.apply(lambda row: get_vegetation_index(row['InitialLongitude'], row['InitialLatitude'], row['FireDiscoveryDateTime']), axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [64]:
def get_mfri(longitude, latitude):
    # Define the dataset
    dataset = ee.ImageCollection('LANDFIRE/Fire/MFRI/v1_2_0')

    # Filter the dataset by location
    filtered_dataset = dataset.filterBounds(ee.Geometry.Point([longitude, latitude]))

    # Check if the image collection is empty
    if filtered_dataset.size().getInfo() == 0:
        return None

    # Select 'MFRI' and calculate the mean
    mean_mfri = filtered_dataset.select('MFRI').mean()

    # Define a region of interest
    roi = ee.Geometry.Rectangle([longitude-0.01, latitude-0.01, longitude+0.01, latitude+0.01])

    # Reduce the image over the region of interest
    mean_mfri = mean_mfri.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30)

    # Get the mean MFRI value
    if 'MFRI' in mean_mfri.getInfo():
        mean_mfri_value = mean_mfri.get('MFRI').getInfo()
    else:
        # Find the nearest station with data (assuming you have a way to define 'stations')
        stations = Stations()  # Replace with your station data source
        stations = stations.nearby(latitude, longitude)
        station = stations.fetch(1)

        if not station.empty:
            nearby_lon = station.iloc[0]['longitude']
            nearby_lat = station.iloc[0]['latitude']
            nearby_roi = ee.Geometry.Rectangle([nearby_lon-0.01, nearby_lat-0.01, nearby_lon+0.01, nearby_lat+0.01])
            mean_mfri = mean_mfri.reduceRegion(reducer=ee.Reducer.mean(), geometry=nearby_roi, scale=30)
            if 'MFRI' in mean_mfri.getInfo():
                mean_mfri_value = mean_mfri.get('MFRI').getInfo()
            else:
                mean_mfri_value = None  # No data found even at the nearest station
        else:
            mean_mfri_value = None  # No nearby stations found

    return mean_mfri_value

testdf['MFRI'] = testdf.apply(lambda row: get_mfri(row['InitialLongitude'], row['InitialLatitude']), axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
testdf.dropna(subset=['VegetationIndex'], inplace=True)
testdf.dropna(subset=['MFRI'], inplace=True)
testdf = testdf.where(testdf['MFRI'] > 0)
testdf = testdf.reset_index(drop=True)
testdf['TimeSinceFire'] = ((datetime.now() - testdf['FireDiscoveryDateTime']).dt.days)/365
testdf.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI,TimeSinceFire
43,1.5,2019-09-05 23:17:55,WF,34.448900,-118.056300,US-CA,29.6,0.0,15.8,1634.994863,12.481210,4.950685
44,80.0,2022-06-21 16:48:36,WF,33.546326,-116.043868,US-CA,32.9,0.0,12.4,1651.378559,30.118662,2.156164
45,2.0,2018-06-16 16:30:00,WF,34.665230,-118.761900,US-CA,9.3,0.0,25.6,1480.511875,33.322216,6.172603
46,350.0,2019-10-10 19:56:55,WF,33.984818,-117.217944,US-CA,19.9,0.0,13.3,1459.243946,10.675924,4.854795
47,2.0,2017-07-26 00:46:09,WF,41.675830,-123.420700,US-CA,13.1,0.0,11.3,3755.749510,3.719308,7.065753


In [66]:
mtemp = testdf['Temperature'].mean()
mprec = testdf['Precipitation'].mean()
mwind = testdf['WindSpeed'].mean()
mveg = testdf['VegetationIndex'].mean()
mMFRI = testdf['MFRI'].mean()
mtimesince = testdf['TimeSinceFire'].mean()

sdtemp = testdf['Temperature'].std()
sdprec = testdf['Precipitation'].std()
sdwind = testdf['WindSpeed'].std()
sdveg = testdf['VegetationIndex'].std()
sdMFRI = testdf['MFRI'].std()
sdtimesince = testdf['TimeSinceFire'].std()

testdata = pd.DataFrame({
    'Temperature': [mtemp, sdtemp],
    'Precipitation': [mprec, sdprec],
    'WindSpeed': [mwind, sdwind],
    'VegetationIndex': [mveg, sdveg],
    'MFRI': [mMFRI, sdMFRI],
    'TimeSinceFire': [mtimesince, sdtimesince]
}, index=['mean', 'std'])

testdata.head()

,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI,TimeSinceFire
mean,21.278723,0.0,11.600000,1967.003945,16.543931,4.971495
std,6.130605,0.0,6.794179,615.440305,15.725407,1.898616


In [67]:
# Apply the function to each row and handle None values
def apply_weather_stats2(row, i):
    stats = weather_stats(row['Latitude'], row['Longitude'], row['Date'])
    return stats[i] if stats is not None else None

def get_time_since_closest_fire(latitude, longitude, firedf):
    """
    Calculates time since the closest fire in firedf for a given latitude and longitude.
    """
    min_distance = float('inf')
    closest_fire_date = None

    for _, fire_row in firedf.iterrows():
        # Validate latitude before calculating distance
        if not (-90 <= fire_row['InitialLatitude'] <= 90):
            print(f"Invalid latitude in firedf: {fire_row['InitialLatitude']}")
            continue  # Skip this row

        # Calculate distance using geodesic (more accurate for lat/long)
        distance = geodesic((latitude, longitude),
                           (fire_row['InitialLatitude'], fire_row['InitialLongitude'])).km

        if distance < min_distance:
            min_distance = distance
            closest_fire_date = fire_row['FireDiscoveryDateTime']

    if closest_fire_date is not None:
        time_since = (datetime.now() - closest_fire_date).days / 365
    else:
        time_since = None  # Handle cases where no fire is found nearby

    return time_since

In [68]:
# Define the data for each column
testdata2 = pd.DataFrame({
    'Temperature': [None],
    'Precipitation': [None],
    'WindSpeed': [None],
    'VegetationIndex': [None],
    'MFRI': [None],
    'TimeSinceFire': [None] # Add TimeSinceFire column to match testdata
})

# Collect user input
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))
date_str = input("Enter date (YYYY-MM-DD): ")
date = pd.to_datetime(date_str)

Enter latitude: 20
Enter longitude: 110
Enter date (YYYY-MM-DD): 2020-09-04


In [69]:
# Apply functions using the user input
temperature = apply_weather_stats2({'Latitude': latitude, 'Longitude': longitude, 'Date': date}, 0)
precipitation = apply_weather_stats2({'Latitude': latitude, 'Longitude': longitude, 'Date': date}, 1)
wind_speed = apply_weather_stats2({'Latitude': latitude, 'Longitude': longitude, 'Date': date}, 2)
vegetation_index = get_vegetation_index(longitude, latitude, date)
mfri = get_mfri(longitude, latitude)
time_since_fire = get_time_since_closest_fire(latitude, longitude, firedf)

if mfri is None:
    mfri = testdata.loc['mean', 'MFRI']

# Create a new row for testdata
new_row = [temperature, precipitation, wind_speed, vegetation_index, mfri, time_since_fire]
testdata.loc['new_data'] = new_row

testdata.head()

Invalid latitude in firedf: -1764.78333333
Invalid latitude in firedf: 3549.602
Invalid latitude in firedf: 100.188333
Invalid latitude in firedf: 119.0
Invalid latitude in firedf: 94.966


,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI,TimeSinceFire
mean,21.278723,0.0,11.600000,1967.003945,16.543931,4.971495
std,6.130605,0.0,6.794179,615.440305,15.725407,1.898616
new_data,29.500000,NaN,8.100000,NaN,16.543931,8.989041


In [70]:
# Create an empty Series to store the results
result = pd.Series(index=testdata.columns)

# Iterate over each column in the DataFrame
for column in testdata.columns:
    if testdata.loc['std', column] == 0:
        value = 0
    else:
        value = (testdata.loc['new_data', column] - testdata.loc['mean', column]) / testdata.loc['std', column]

    # Add the result to the new DataFrame
    result.loc[column] = value

# Print the result
testdata.loc['stdaway'] = result

testdata.head()

,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI,TimeSinceFire
mean,21.278723,0.0,11.600000,1967.003945,16.543931,4.971495
std,6.130605,0.0,6.794179,615.440305,15.725407,1.898616
new_data,29.500000,NaN,8.100000,NaN,16.543931,8.989041
stdaway,1.341022,0.0,-0.515147,NaN,0.000000,2.116040


In [71]:
import math

fire_danger_1 = 2*testdata.loc['stdaway', 'Temperature'] - testdata.loc['stdaway', 'Precipitation'] \
                + testdata.loc['stdaway', 'WindSpeed'] + testdata.loc['stdaway', 'VegetationIndex'] \
                - testdata.loc['stdaway', 'MFRI']/3
fire_danger = 10/(1+math.exp(-1*fire_danger_1))
print(fire_danger)

nan


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assume df is your DataFrame and "wildfire_size" is the target variable
X = testdf[['Temperature', 'Precipitation', 'WindSpeed', 'VegetationIndex', 'MFRI', 'TimeSinceFire']]
y = testdf["IncidentSize"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2003)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [73]:
#not doing anything useful yet

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the new data for a few epochs
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)  # Adjust the number of epochs and batch size as needed


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: nan - val_loss: nan
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: nan - val_loss: nan
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: nan - val_loss: nan
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: nan - val_loss: nan
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: nan - val_loss: nan
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: nan - val_loss: nan
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: nan - val_loss: nan
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: nan - val_loss: nan
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: nan - val_loss: nan
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: nan - val_loss: nan
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: nan - val_loss: nan
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: nan - val_loss: nan
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: na